This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [72]:
# First, import the relevant modules
import requests
from collections import namedtuple


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json')

In [43]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
#print(r.json())

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1) Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

Quandl allows one to filter results based on start and end date. Here, the start_date and end_date parameters are passed in to the get request

In [20]:
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=%s&start_date=2017-01-01&end_date=2017-12-31' % API_KEY)

2) Convert the returned JSON object into a Python dictionary.

In [25]:
afx_x = r.json()
print(type(afx_x))

<class 'dict'>


Convert the data to a more useful dict: dictionary with date as key and namedTuple with daily data as value.

In [44]:
# namedtuple cannot have spaces in *names*. Instead, replace spaces with underscore for column names
col_names = [mystr.replace(" ", "_") for mystr in afx_x['dataset_data']['column_names'][1:]]
TradeDate = namedtuple('TradeDate', col_names)

#loop through all of the data and make date the key and tuple of rest of data as value
afx = {daily_value[0]:TradeDate._make(daily_value[1:]) 
       for daily_value in afx_x['dataset_data']['data']}
#for daily_value in afx_x['dataset_data']['data'][:10]:
#    afx[daily_value[0]] = TradeDate._make(daily_value[1:])

In [46]:
print(col_names)

['Open', 'High', 'Low', 'Close', 'Change', 'Traded_Volume', 'Turnover', 'Last_Price_of_the_Day', 'Daily_Traded_Units', 'Daily_Turnover']


In [45]:
print(afx['2017-12-29'])
print(datetime.strptime())

TradeDate(Open=51.76, High=51.94, Low=51.45, Close=51.76, Change=None, Traded_Volume=34640.0, Turnover=1792304.0, Last_Price_of_the_Day=None, Daily_Traded_Units=None, Daily_Turnover=None)


3) Calculate what the highest and lowest opening prices were for the stock in this period.


In [65]:
opening_prices = [v.Open for v in afx.values() if v.Open != None]
print("Highest opening price: $%.2f" % max(opening_prices))
print("Lowest  opening price: $%.2f" % min(opening_prices))

Highest opening price: $53.11
Lowest  opening price: $34.00


4) What was the largest change in any one day (based on High and Low price)?


In [83]:
change = [abs(v.High - v.Low) for v in afx.values() if v.High != None and v.Low != None]
print("Largest Change in one day: $%.2f" % max(change))

Largest Change in one day: $2.81


5) What was the largest change between any two days (based on Closing Price)?


In [84]:
# I am using python version >=3.6, so all dicts are orderedDicts (remember the order of items inserted)
# Items were inserted as latest date first
# Will assume that 2 days = 2 trading days (i.e. Friday - Monday are sequential days)

# keep track of largest change
largest_change = 0.0
#loop through all keys, 
for a, (k, v) in enumerate(afx.items()):
    # do not compare if first key
    if a > 0:
        # get change between previous key close price and current key close price
        new_change = abs(afx[old_key].Close - v.Close)
        if new_change > largest_change:
            largest_change = new_change
    old_key = k
print("Largest Change in two days: $%.2f" % largest_change)

Largest Change in two days: $2.56


6) What was the average daily trading volume during this year?


In [92]:
traded_volume = [v.Traded_Volume for v in afx.values() if v.Traded_Volume != None]
print("Avg Traded Volume: %.2f" % (sum(traded_volume)/len(traded_volume)) )

Avg Traded Volume: 89124.34


7) (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [101]:
# define function for getting median in list
def median(vals):
    # return middle value in a sorted list
    return sorted(vals)[int(len(vals)/2)]

In [105]:
print("Median Traded Volume: %.f" % median(traded_volume))

Median Traded Volume: 76286
